# Init

In [59]:
# init
from importlib import reload
from pprint import pprint
import os
import sys
from pathlib import Path
import pandas as pd

def reloadPckg():
    reload(tgm)
    reload(too)
    reload(osmt)

sys.path.append('..')
sys.path.append('../tools')
devPath = "C:/Users/gonta/D/software developer/"
sys.path.append(os.path.join(devPath, "packages"))


import tests.osm_test as osmt
import toolsGeneral.main as tgm
import toolsOSM.overpass as too
from toolsPandas.helpers import df_peek 

In [40]:
# init
import tools.logger_config as logconfig

# Cleaning

## * import

In [17]:
dataDir = Path(os.path.join(os.getcwd(), '..', 'data', 'raw', 'osm countries queries'))
rawFiles = [f for f in dataDir.rglob('*.json')]

rawByCntr = {
    file.parent.name:tgm.load(str(file))['elements']
    for file in rawFiles
}
print(len(rawByCntr))

76


In [2]:
pprint({k:len(v) for k,v in rawByCntr.items()})

{'Albania': 390,
 'Algeria': 2148,
 'Andorra': 1,
 'Australia': 616,
 'Austria': 2178,
 'Austria--Liechtenstein': 1,
 'Austria-Switzerland': 1,
 'Bangladesh': 453,
 'Belarus': 1431,
 'Belgium': 4351,
 'Bhutan': 1191,
 'Brazil': 792,
 'Bulgaria': 113,
 'Cambodia': 316,
 'Chile': 420,
 'Colombia': 3806,
 'Czechia': 6281,
 'Denmark': 6,
 'Ecuador': 1240,
 'Estonia': 1,
 'Eswatini': 58,
 'FaroeIslands': 1,
 'Finland': 20,
 'France-Andorra': 1,
 'France-Belgium': 1,
 'France-Germany': 1,
 'France-Italy': 1,
 'France-Luxembourg': 1,
 'France-Monaco': 1,
 'France-Spain': 1,
 'France-Switzerland': 1,
 'Georgia': 163,
 'Germany': 11264,
 'Germany--Switzerland': 1,
 'Germany-Austria': 1,
 'Greece': 589,
 'Guinea': 90,
 'Guinea-bissau': 47,
 'Hungary': 3179,
 'Ireland': 1,
 'IsleOfMan': 1,
 'Jordan': 106,
 'Kyrgyzstan': 68,
 'Laos': 170,
 'Latvia': 1,
 'Lebanon': 36,
 'Libya': 24,
 'Lithuania': 4997,
 'Mauritania': 26,
 'Mexico': 3129,
 'Moldova': 38,
 'Mongolia': 363,
 'Montenegro': 1,
 'Myanmar

## * convert id to string

In [38]:
for k,v in rawByCntr.items():
    for ele in v:
        ele['id'] = str(ele['id'])

## * use only sovereign countries

In [39]:
sovereign_countries = tgm.load(os.path.join('..', 'data', 'sovereign countries.json'))
len(sovereign_countries)

214

In [40]:
df_by_cntr = {k:elems for k,elems in rawByCntr.items() if k in sovereign_countries}
len(df_by_cntr)

64

## * add parent_id to level 4 entities

In [41]:
temp = [ele['tags'].get('parent_id',None) for k,v in df_by_cntr.items() for ele in v]
pd.Series(temp).map(type).value_counts()

<class 'str'>         118674
<class 'NoneType'>        64
Name: count, dtype: int64

In [42]:
for k,v in df_by_cntr.items():
    cntr_id = list(filter(lambda ele: ele['tags']['admin_level'] == '2', v))[0]
    cntr_id = cntr_id['id']
    for ele in v:
        if ele['tags']['admin_level'] == '4':
            ele['tags']['parent_id'] = str(cntr_id)

## * add country_name and country_id tag

In [43]:
for k,v in df_by_cntr.items():
    for ele in v:
        ele['tags']['country_name'] = k
for k,v in df_by_cntr.items():
    cntr_ele = list(filter(lambda ele: ele['tags']['admin_level'] == '2', v))[0]
    for ele in v:
        ele['tags']['country_id'] = cntr_ele['id']

## * normalize

In [44]:
df_by_cntr = {k:too.normalizeOSM(elems) for k,elems in df_by_cntr.items()}
len(df_by_cntr)

64

In [62]:
combined = pd.concat(df_by_cntr.values(), ignore_index=True)
combined.map(type).stack().value_counts()

<class 'pandas._libs.missing.NAType'>    940391
<class 'str'>                            840679
Name: count, dtype: int64

In [63]:
for k,df in df_by_cntr.items():
    cntr = df[df['tags.admin_level']=='2']
    print([k, cntr['tags.name'].iloc[0]])

['Albania', 'Shqipëria']
['Algeria', 'Algérie ⵍⵣⵣⴰⵢⴻⵔ الجزائر']
['Andorra', 'Andorra']
['Australia', 'Australia']
['Austria', 'Österreich']
['Bangladesh', 'বাংলাদেশ']
['Belarus', 'Беларусь']
['Belgium', 'België / Belgique / Belgien']
['Bhutan', 'འབྲུག་ཡུལ།']
['Brazil', 'Brasil']
['Bulgaria', 'България']
['Cambodia', 'ព្រះរាជាណាចក្រ\u200bកម្ពុជា']
['Chile', 'Chile']
['Colombia', 'Colombia']
['Czechia', 'Česko']
['Denmark', 'Danmark']
['Ecuador', 'Ecuador']
['Estonia', 'Eesti']
['Eswatini', 'Eswatini']
['FaroeIslands', 'Føroyar']
['Finland', 'Suomi / Finland']
['Georgia', 'საქართველო']
['Germany', 'Deutschland']
['Greece', 'Ελλάς']
['Guinea', 'Guinée']
['Guinea-bissau', 'Guiné-Bissau']
['Hungary', 'Magyarország']
['Ireland', 'Éire / Ireland']
['IsleOfMan', 'Isle of Man']
['Jordan', 'الأردن']
['Kyrgyzstan', 'Кыргызстан']
['Laos', 'ປະເທດລາວ']
['Latvia', 'Latvija']
['Lebanon', 'لبنان']
['Libya', 'ليبيا']
['Lithuania', 'Lietuva']
['Mauritania', 'Mauritanie موريتانيا']
['Mexico', 'México']
['

## * SAVE!

In [64]:
# save data by country
for k,df in df_by_cntr.items():
    path = os.path.join('../data/normalized', k, f'{k}_normalized.pkl')
    tgm.dump(path, df)

# Test elems

In [1]:
# import
dataDir = Path(os.path.join(os.getcwd(), '..', 'data/normalized'))
rawFiles = [f for f in dataDir.rglob('*.pkl')]
print(len(rawFiles))

df_by_cntr = {
    file.parent.name:tgm.load(str(file))
    for file in rawFiles
}
print(len(df_by_cntr))

64
64


## 1. basic elems test

In [4]:
def test(cntr, df):
    print(cntr)
    return osmt.osm_basic_test(df)

osm_basic_test = {cntr: test(cntr, df) for cntr, df in df_by_cntr.items()}

Albania
 * missing names:  0
 * relations from other countries:  0
Algeria
 * missing names:  0
 * relations from other countries:  0
Andorra
 * missing names:  0
 * relations from other countries:  0
Australia
 * missing names:  0
 * relations from other countries:  0
Austria
 * missing names:  0
 * relations from other countries:  0
Bangladesh
 * missing names:  1
 * relations from other countries:  0
Belarus
 * missing names:  0
 * relations from other countries:  0
Belgium
 * missing names:  0
 * relations from other countries:  7
Bhutan
 * missing names:  0
 * relations from other countries:  0
Brazil
 * missing names:  0
 * relations from other countries:  2
Bulgaria
 * missing names:  0
 * relations from other countries:  0
Cambodia
 * missing names:  0
 * relations from other countries:  0
Chile
 * missing names:  0
 * relations from other countries:  1
Colombia
 * missing names:  8
 * relations from other countries:  0
Czechia
 * missing names:  0
 * relations from other count

### # review

In [24]:
temp = df_by_cntr['Russia']
temp[temp['id'].isin(['4638816','60189'])]

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3,tags.addr:country
0,relation,60189,2,<NA>,Россия,<NA>,RU,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3446,rel,4638816,8,19162030,Клепининское сельское поселение,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,RU
3848,rel,4638816,8,1754554,Клепининское сельское поселение,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,RU


In [25]:
osmt.osm_basic_test(temp[temp['id'].isin(['4638816','60189'])])

 * missing names:  0
['RU', 'RU']
['RU', 'RU']
 * relations from other countries:  0


{'missing.name': [], 'leak': []}

In [ ]:
# tgm.dump(os.path.join('..', 'data', 'tests results', 'osm_basic_test.pkl'), osm_basic_test)

In [2]:
osm_basic_test = tgm.load(os.path.join('..', 'data', 'tests results', 'osm_basic_test.pkl'))
len(osm_basic_test)

64

In [18]:
temp = pd.DataFrame(osm_basic_test)
df_peek(temp.T)

,missing.name,leak
Albania,[],[]
Algeria,[],[]
Andorra,[],[]
Australia,[],[]
Austria,[],[52345]
Bangladesh,[14326679],"[3824588, 3825003, 3859335, 3921211, 3921222, 3921298, 3921322, 7318343, 9455570]"
Belarus,[],"[59162, 59189, 59195, 59275, 59506, 59752]"
Belgium,[],"[4217435, 7394, 7400, 7411, 7427, 7463, 7400]"
Bhutan,[],[]
Brazil,[],"[54882, 57963, 185579, 242620, 298204, 301079, 301464, 302635, 302819, 303702, 303781, 303940, 315173, 325866, 326266, 331463, 332476, 332924, 333597, 334051, 334443, 336819, 362413, 405935, 421151, 4059665, 298424, 298446]"


In [83]:
df_peek(temp.T.map(len))

,missing.name,leak
Albania,0,0
Algeria,0,0
Andorra,0,0
Australia,0,0
Austria,0,1
Bangladesh,1,9
Belarus,0,6
Belgium,0,7
Bhutan,0,0
Brazil,0,28


In [15]:
to_delete_basic = []
for k,v in osm_basic_test.items():
    to_delete_basic.extend(v['missing.name'])
    to_delete_basic.extend(v['leak'])
temp = tgm.count_duplicates(to_delete_basic)
print(len(temp))

to_delete_basic = set(to_delete_basic)
print(len(to_delete_basic))

240
292


## 2. osm leaked elem center inside country test

In [41]:
combined_df = pd.concat(df_by_cntr.values(), ignore_index=True)
len(combined_df)

118738

In [42]:
dupplicates = combined_df[combined_df.duplicated("id", keep=False)]
len(dupplicates)

1636

## 3. osm duplicates test

### * load processed data

In [48]:
odt_processed_info = tgm.load(os.path.join(os.getcwd(), "../data/tests results/osm duplicates test", "odt_processed_info.pkl"))
len(odt_processed_info)

1341

In [50]:
combined_df = pd.concat(df_by_cntr.values(), ignore_index=True)
len(combined_df)

118738

### * combine dataframes and find duplicates

In [54]:
combined_df = pd.concat(df_by_cntr.values(), ignore_index=True)
len(combined_df)

118738

In [57]:
dups_df = combined_df[combined_df.duplicated("id", keep=False)]
print(len(dups_df))
dups_pairs = list(map(tuple, dups_df[['id','tags.parent_id']].values))
print(len(dups_pairs))
dups_pairs = set(dups_pairs)
print(len(dups_pairs))

1636
1636
946


In [29]:
pairs_already_tested = [(ele[1],ele[2]) for ele in odt_processed_info]
print(len(pairs_already_tested))
pairs_already_tested = set(pairs_already_tested)
print(len(pairs_already_tested))

1341
791


In [46]:
('4496067', '174737') in pairs_already_tested

False

In [51]:
combined_pairs = combined_df[['id','tags.parent_id']].apply(tuple,axis=1).to_list()

In [52]:
('4496067', '174737') in combined_pairs

True

In [111]:
tgm.count_duplicates(combined_pairs)

{('1845173', '4496067'): 2,
 ('1846634', '4496067'): 2,
 ('5305784', '4496067'): 2,
 ('5305896', '4496067'): 2,
 ('5323564', '4496067'): 2,
 ('5323577', '4496067'): 2,
 ('5323612', '4496067'): 2,
 ('5323628', '4496067'): 2,
 ('5323643', '4496067'): 2,
 ('5323832', '4496067'): 2,
 ('5324073', '4496067'): 2,
 ('5324074', '4496067'): 2,
 ('5324075', '4496067'): 2,
 ('5324202', '4496067'): 2,
 ('5324235', '4496067'): 2,
 ('5324258', '4496067'): 2,
 ('5324274', '4496067'): 2,
 ('11274601', '4496067'): 2,
 ('1613889', '223136'): 2,
 ('1624125', '223136'): 2,
 ('1645432', '223136'): 2,
 ('1827275', '223136'): 2,
 ('1827276', '223136'): 2,
 ('1827293', '223136'): 2,
 ('1827294', '223136'): 2,
 ('1827295', '223136'): 2,
 ('1827377', '223136'): 2,
 ('1827378', '223136'): 2,
 ('1827399', '223136'): 2,
 ('1827656', '223136'): 2,
 ('3671798', '223136'): 2,
 ('5324202', '223136'): 2,
 ('19258534', '223136'): 2,
 ('8324061', '5323612'): 2,
 ('8325070', '5323612'): 2,
 ('16807920', '5324274'): 2,
 ('1

In [11]:
combined_df[combined_df['id']=='1845173']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3,tags.addr:country,tags.country_name,tags.country_id
37261,rel,1845173,6,4496067,Περιφερειακή Ενότητα Σάμου,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Greece,192307
99869,rel,1845173,6,4496067,Περιφερειακή Ενότητα Σάμου,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Turkey,174737


In [8]:
combined_df[combined_df['id']=='4496067']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3,tags.addr:country,tags.country_name,tags.country_id
37246,relation,4496067,4,192307,Αποκεντρωμένη Διοίκηση Αιγαίου,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Greece,192307
99868,relation,4496067,4,174737,Αποκεντρωμένη Διοίκηση Αιγαίου,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Turkey,174737


In [2]:
# greece: 192307
# turkey: 174737
dup_test_logger = logconfig.initiate_logger('../data/tests results/osm duplicates test/odt_dup_test.log', 'dup_test_logger')

In [13]:
osmt.getCenterNodeInsideParent('4496067', '174737',dup_test_logger)

[INFO] :    * Getting center of child: 
[INFO] :    * Getting parent that contains center: 


{'status': 'ok',
 'data': {'version': 0.6,
  'generator': 'Overpass API 0.7.62.8 e802775f',
  'osm3s': {'timestamp_osm_base': '2025-10-31T21:00:00Z',
   'timestamp_areas_base': '2025-10-31T12:29:30Z',
   'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
  'elements': []}}

In [5]:
dups_df = combined_df[combined_df.duplicated("id", keep=False)]
print(len(dups_df))
dups_pairs = list(map(tuple, dups_df[['id','tags.parent_id']].values))
print(len(dups_pairs))
dups_pairs = set(dups_pairs)
print(len(dups_pairs))

1636
1636
946


In [7]:
dups_pairs_to_test = tgm.complement(dups_pairs, pairs_already_tested)
print(len(dups_pairs_to_test))

155


In [6]:
combined_df[combined_df['id']=='223136']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3,tags.addr:country,tags.country_name
37241,relation,223136,4,192307,Muğla,<NA>,<NA>,TR-48,<NA>,TR323,<NA>,TR323,<NA>,Greece
99805,relation,223136,4,174737,Muğla,<NA>,<NA>,TR-48,<NA>,TR323,<NA>,TR323,<NA>,Turkey


In [4]:
osmt.getCenterNodeInsideParent('223136','192307',dup_test_logger)

[INFO] :    * Getting center of child: 
[INFO] :    * Getting parent that contains center: 
[INFO] :    * Attempt 1 failed: http_error: 504 Server Error: Gateway Timeout for url: http://overpass-api.de/api/interpreter?data=%0A++++++++%5Bout%3Ajson%5D%5Btimeout%3A300%5D%3B%0A%0A++++++++is_in%2836.9184001%2C+28.455736%29-%3E.areas%3B%0A++++++++rel%28pivot.areas%29%28id%3A192307%29%3B%0A++++++++out+ids%3B%0A++++


{'status': 'ok',
 'data': {'version': 0.6,
  'generator': 'Overpass API 0.7.62.8 e802775f',
  'osm3s': {'timestamp_osm_base': '2025-10-31T20:26:01Z',
   'timestamp_areas_base': '2025-10-31T13:27:29Z',
   'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
  'elements': []}}

In [8]:
df_by_cntr_to_test = {}
for k,df in df_by_cntr.items():
    elems = df[['id','tags.parent_id']].apply(tuple, axis=1).isin(dups_pairs_to_test)
    elems = df[elems]
    if not elems.empty:
        df_by_cntr_to_test[k] = elems

In [51]:
df_by_cntr_to_test.keys()

dict_keys(['Greece', 'Lebanon', 'Libya', 'Russia', 'Syria', 'Tunisia', 'Turkey', 'Ukraine', 'Vietnam'])

In [52]:
df_by_cntr_to_test['Ukraine']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3,tags.addr:country,tags.country_name
19,relation,72639,4,60199,Автономна Республіка Крим,<NA>,<NA>,UA-43,<NA>,<NA>,<NA>,<NA>,<NA>,Ukraine
27,relation,1574364,4,60199,Севастополь,<NA>,<NA>,UA-40,Ukraine,<NA>,<NA>,<NA>,UA,Ukraine
28,relation,3788485,4,60199,Севастополь,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,RU,Ukraine
72,rel,1574582,6,72639,Бахчисарайский район,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,UA,Ukraine
73,rel,1590251,6,72639,Черноморский район,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ukraine
74,rel,1754552,6,72639,Белогорский район,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ukraine
75,rel,1754553,6,72639,Джанкойский район,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ukraine
76,rel,1754554,6,72639,Красногвардейский район,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ukraine
77,rel,1754555,6,72639,Красноперекопский район,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ukraine
78,rel,1754556,6,72639,Кировский район,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ukraine


### * run test

In [94]:
odt_processed_chunks = tgm.load(os.path.join(os.getcwd(), f"../data/tests results/osm duplicates test/chunk_test_res.pkl"))
len(odt_processed_chunks)

1

In [101]:
temp = pd.concat(df_by_cntr_to_test.values(), ignore_index=True)
len(temp)

294

In [99]:
len(dups_pairs_to_test)

155

In [ ]:
dup_test_logger = logconfig.initiate_logger('../data/tests results/osm duplicates test/odt_dup_test.log', 'dup_test_logger')

odt_test_res = osmt.countries_run_test(
    df_by_cntr_to_test, 
    osmt.osm_duplicates_test_center,
    dup_test_logger,
    save=False,
    save_dir=os.path.join(os.getcwd(), f"../data/tests results/osm duplicates test"),
    processed_info = odt_processed_info,
    test_pairs=True
)

[INFO] : [1/9] Processing Greece
[INFO] :   * chunk size 2: current [2/42]
[INFO] :  - duplicate elements by id: 2
[INFO] :  - testing osm center: 2
[INFO] :   * [1/2]: testing ['1845173', '4496067']:
[INFO] :    * Getting center of child: 
[INFO] :    * Attempt 1 failed: http_error: 429 Client Error: Too Many Requests for url: http://overpass-api.de/api/interpreter?data=%0A++++++++%5Bout%3Ajson%5D%5Btimeout%3A300%5D%3B%0A%0A++++++++rel%281845173%29%3B%0A++++++++out+center%3B%0A++++


KeyboardInterrupt: 

In [ ]:
odt_test_res_not_empty = {
    k:v 
    for k,v in odt_test_res.items()
    if v['test_res']
    and v['duplicate.keep_elems']
    and v['duplicate.delete_elems']
}
len(odt_test_res_not_empty)

1

### * merge and save results

In [70]:
test_res_to_merge = odt_test_res_not_empty
print(len(test_res_to_merge))

1


In [71]:
for k,v in test_res_to_merge.items():
    data_path = os.path.join(os.getcwd(), f"../data/tests results/osm duplicates test/{k}/{k}_test_res.pkl")
    if not os.path.exists(data_path):
        tgm.dump(data_path, v)
    else:
        test_res_old_value = tgm.load(data_path)
        test_res_to_save= {}
        test_res_to_save['test_res'] = test_res_old_value['test_res'] | test_res_to_merge[k]['test_res']
        test_res_to_save['duplicate.keep_elems'] = list(set(test_res_old_value['duplicate.keep_elems']) | set(test_res_to_merge[k]['duplicate.keep_elems']))
        test_res_to_save['duplicate.delete_elems'] = list(set(test_res_old_value['duplicate.delete_elems']) | set(test_res_to_merge[k]['duplicate.delete_elems']))
        tgm.dump(data_path, test_res_to_save)

In [51]:
# Load old results
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))
odt_test_res_files = [f for f in data_dir.rglob('*/*.pkl')]
print(len(odt_test_res_files))

odt_test_res_old = {
    file.parent.name:tgm.load(str(file))
    for file in odt_test_res_files
}

64


In [53]:
odt_test_res_old['Vietnam']['test_res']

{('19415099', '1844412'): {'status': 'ok',
  'result': True,
  'data': {'version': 0.6,
   'generator': 'Overpass API 0.7.62.8 e802775f',
   'osm3s': {'timestamp_osm_base': '2025-10-26T17:49:55Z',
    'timestamp_areas_base': '2025-10-26T11:58:06Z',
    'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
   'elements': [{'type': 'relation', 'id': 1844412}]}},
 ('19415099', '1902967'): {'status': 'ok',
  'result': False,
  'data': {'version': 0.6,
   'generator': 'Overpass API 0.7.62.8 e802775f',
   'osm3s': {'timestamp_osm_base': '2025-10-26T17:49:55Z',
    'timestamp_areas_base': '2025-10-26T11:58:06Z',
    'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
   'elements': []}},
 ('12138923', '19539163'): {'status': 'ok',
  'result': False,
  'data': {'version': 0.6,
   'generator': 'Overpass API 0.7.62.8 e802775f',
   'osm3s': {'timestamp_osm_base

### * resume

In [ ]:
# Resume
temp = pd.DataFrame(odt_test_res_old)
df_peek(temp.map(len).T)

,test_res,duplicate.keep_elems,duplicate.delete_elems
Albania,0,0,0
Algeria,2,1,1
Andorra,0,0,0
Australia,0,0,0
Austria,0,0,0
Bangladesh,4,2,2
Belarus,10,5,5
Belgium,8,4,4
Bhutan,2,1,1
Brazil,8,3,5


In [2]:
to_delete_dup = []
to_keep_elems = []
for k,v in odt_test_res_old.items():
    to_delete_dup.extend(v['duplicate.delete_elems'])
    to_keep_elems.extend(v['duplicate.keep_elems'])

to_delete_dup = set(to_delete_dup)
print(len(to_delete_dup))
to_keep_elems = set(to_keep_elems)
print(len(to_keep_elems))


143
648


### * check

In [26]:
dups_pairs_to_test = tgm.complement(dups_pairs, already_tested)
print(len(dups_pairs_to_test))

294


In [36]:
rls_to_test = {}
for c,df in df_by_cntr.items():
    temp_df = df[df[['id', 'tags.parent_id']].apply(tuple,axis=1).isin(dups_pairs_to_test)]
    if not temp_df.empty:
        rls_to_test[c] = temp_df

In [49]:
rls_to_test.keys()

dict_keys(['Greece', 'Lebanon', 'Libya', 'Russia', 'Syria', 'Tunisia', 'Turkey', 'Ukraine', 'Vietnam'])

In [60]:
len(pd.concat(rls_to_test.values(), ignore_index=True))

294

In [109]:
temp1 = tgm.complement(dups_pairs, to_delete_dup)
print(len(temp1))
temp2 = tgm.complement(temp1, to_keep_elems)
print(len(temp2))

1478
294


In [123]:
osmt.osm_duplicates_test_center(combined[combined['id']=='223136'])

 - Getting center of child: 
 - Getting parent that contains center: 
 - Getting center of child: 
 - Getting parent that contains center: 


{'test_res': {('223136', '192307'): {'status': 'ok',
   'result': False,
   'data': {'version': 0.6,
    'generator': 'Overpass API 0.7.62.8 e802775f',
    'osm3s': {'timestamp_osm_base': '2025-10-30T19:22:45Z',
     'timestamp_areas_base': '2025-10-30T12:08:45Z',
     'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
    'elements': []}},
  ('223136', '174737'): {'status': 'ok',
   'result': True,
   'data': {'version': 0.6,
    'generator': 'Overpass API 0.7.62.8 e802775f',
    'osm3s': {'timestamp_osm_base': '2025-10-30T19:22:45Z',
     'timestamp_areas_base': '2025-10-30T12:08:45Z',
     'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
    'elements': [{'type': 'relation', 'id': 174737}]}}},
 'duplicate.keep_elems': [('223136', '174737')],
 'duplicate.delete_elems': [('223136', '192307')]}

In [125]:
combined[combined['id']=='223136']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3,tags.addr:country
37241,relation,223136,4,192307,Muğla,<NA>,<NA>,TR-48,<NA>,TR323,<NA>,TR323,<NA>
99805,relation,223136,4,174737,Muğla,<NA>,<NA>,TR-48,<NA>,TR323,<NA>,TR323,<NA>


In [ ]:
('223136','192307')in dups_pairs

True

In [128]:
list(odt_test_res_old['Turkey']['test_res'].keys())

[('5324202', '4496067'),
 ('5324202', '223136'),
 ('11628545', '1211496'),
 ('11629050', '1211496'),
 ('11628545', '1211488'),
 ('11629050', '1211488'),
 ('8332387', '1798306'),
 ('8332387', '1731539')]

In [120]:
test_res_joined = {}
for c,v in odt_test_res_old.items():
    test_res_joined.update(v['test_res'])
len(test_res_joined)

793

## compute new odt_processed_info

In [74]:
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))
odt_test_res_files = [f for f in data_dir.rglob('*/*.pkl')]
print(len(odt_test_res_files))

odt_processed_info_new = {}
for file in odt_test_res_files:
    test_res = tgm.load(str(file))['test_res']
    for k,v in test_res.items():
        if v['status'] == 'ok':
            odt_processed_info_new[(file.parent.name, k[0], k[1])] = dict(filter(lambda item: item[0] in ['status','result'], v.items()))

64


In [75]:
tgm.dump(os.path.join(os.getcwd(), "../data/tests results/osm duplicates test", "odt_processed_info.pkl"), odt_processed_info_new)